# 0. Meta

## 0.1. Packages

In [48]:
import pandas as pd
from datetime import datetime
import numpy as np
import os
import joblib
import optuna
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, HuberRegressor, QuantileRegressor, RANSACRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.neighbors import KNeighborsRegressor
#from sklearn.svm import SVR
from sklearn.svm import LinearSVR

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import KFold, cross_val_predict
import logging
import optuna

optuna.logging.set_verbosity(optuna.logging.INFO)

## 0.2. Functions

# 1. Data Import

In [7]:
X_test = pd.read_csv("../data/processed/X_test.csv")
y_test = pd.read_csv("../data/processed/y_test.csv")
X_train = pd.read_csv("../data/processed/X_train.csv")
y_train = pd.read_csv("../data/processed/y_train.csv")

# 2. Model Training and Hyperparameter Tuning

## 2.1. Linear Regression

Train linear regression model and measure computation time.

In [8]:
lr_model = LinearRegression() 
lr_start = datetime.now()
lr_model.fit(X_train, y_train)
lr_pred = lr_model.predict(X_test)
lr_stop = datetime.now()
lr_delta = lr_stop - lr_start

Due to the log-transformation applied to the target variable y_train during data preprocessing to ensure normality and linearity, it is necessary to reverse this transformation on the predictions before assessing the model using metrics such as RMSE and R-squared.

In [9]:
y_test_log = np.log1p(y_test.to_numpy())

lr_pred_df = pd.DataFrame({'pred': lr_pred.flatten(), 'y_test': y_test_log.flatten()})

lr_pred_df['pred'] = np.expm1(lr_pred_df['pred'])
lr_pred_df['y_test'] = np.expm1(lr_pred_df['y_test'])

c:\Sync\03_projects\data_science\projects\autoscout24-car-price-prediction\envs\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: overflow encountered in expm1
  result = getattr(ufunc, method)(*inputs, **kwargs)


During the back-transformation, NaNs and Infs occurred for some rows. These entries are eliminated, and the indices of the removed rows are stored to identify the observations that led to these issues. Additionally, the total count and the relative proportion of dropped rows are computed.

In [10]:
index_before = lr_pred_df.index
rows_before = lr_pred_df.shape[0]

lr_pred_df = lr_pred_df.replace([np.inf, -np.inf], np.nan).dropna()

index_after = lr_pred_df.index
rows_after = lr_pred_df.shape[0]

removed_indices = index_before.difference(index_after)
removed_rows = rows_before - rows_after
percent_removed = (removed_rows / rows_before) * 100

removed_rows_X_test = X_test.iloc[removed_indices]
removed_rows_y_test = y_test.iloc[removed_indices]
removed_rows_df = pd.concat([removed_rows_X_test, removed_rows_y_test], axis=1)
removed_rows_df['pre_backtrans_pred'] = lr_pred.flatten()[removed_indices]
filtered_removed_rows_df = removed_rows_df.loc[:, (removed_rows_df != 0).any()]

print('Rows before removing NaNs and Infs:', rows_before)
print('Rows after removing NaNs and Infs:', rows_after)
print('Number of rows removed:', removed_rows)
print('Percentage of rows removed:', percent_removed, '%')

print("Problematic rows in X_test and y_test:")
filtered_removed_rows_df

Rows before removing NaNs and Infs: 9215
Rows after removing NaNs and Infs: 9193
Number of rows removed: 22
Percentage of rows removed: 0.23874118285404233 %
Problematic rows in X_test and y_test:


,mileage,offerType,hp,year,make_Audi,make_BMW,make_Bentley,make_Corvette,make_DS,make_Fiat,...,model_S7,model_SLC 250,model_T5 Shuttle,fuel_Diesel,fuel_Electric,fuel_Gasoline,gear_Automatic,gear_Manual,price,pre_backtrans_pred
77,0.139248,0.50,0.538058,1.0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,83870,3.996071e+09
101,0.300000,0.00,0.732819,0.6,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,34950,5.998644e+09
824,0.406800,0.00,0.339380,0.5,0,0,0,0,1,0,...,0,0,0,0,0,1,0,1,8500,7.560187e+09
1032,0.283372,0.00,0.678830,0.7,0,1,0,0,0,0,...,0,0,0,0,0,1,1,0,35000,3.379601e+08
2314,0.403326,0.00,0.810958,0.0,0,0,1,0,0,0,...,0,0,0,0,0,1,1,0,99800,3.948626e+10
2468,0.126833,0.00,0.770530,0.8,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,86885,3.379601e+08
2550,0.554074,0.00,0.493344,0.4,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,9985,5.998644e+09
3623,0.398366,0.00,0.744444,0.3,1,0,0,0,0,0,...,1,0,0,0,0,1,1,0,39925,7.201592e+09
3684,0.208001,0.50,0.538058,1.0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,45555,6.188076e+09
4405,0.020801,0.75,0.409779,1.0,0,0,0,0,0,1,...,0,0,0,0,1,0,1,0,33994,7.184330e+09


It appears that the linear regression model leads to extremely high predicted values for the problematic rows. Therefore, the back transformation fails, leading to infinite values. The full linear regression model therefore appears to be unsuitable. After removing the problematic rows, the evaluation metrics are calculated.  However, it should be noted that these can only be interpreted to a limited extent, as not all predictions are taken into account.

In [11]:
lr_r2 = r2_score(lr_pred_df['y_test'], lr_pred_df['pred'])
lr_r2_adj = 1 - (1 - lr_r2) * ((len(X_test) - 1) / (len(X_test) - len(X_test.columns) - 1))
lr_rmse = np.sqrt(mean_squared_error(lr_pred_df['y_test'], lr_pred_df['pred']))
lr_seconds = lr_delta.seconds + lr_delta.microseconds/1E6

lr_evaluation = pd.DataFrame({
    'model': ['lr'],
    'r2': [lr_r2],
    'r2_adj': [lr_r2_adj],
    'rmse': [lr_rmse],
    'seconds': [lr_seconds]
})

lr_evaluation

,model,r2,r2_adj,rmse,seconds
0,lr,0.928132,0.920122,4906.257588,8.711491


Save the trained model, model predictions and evaluation metrics.

In [10]:
joblib.dump(lr_model, '../models/models/lr_model.pkl')
lr_evaluation.to_csv(os.path.join("../models/evaluation/", "lr_evaluation.csv"), index=False)
lr_res = pd.DataFrame(lr_pred)
lr_res.index = X_test.index
lr_res.columns = ["prediction"]
lr_res.to_csv("../models/predictions/lr_prediction.csv")

## 2.2. Regularized Linear Regression

### 2.2.1. Lasso Regression

Start the hyperparameter optimization with an initial rough exploration of the hyperparameter space using Optuna's TPESampler algorithm (Tree-Structured Parzen Estimator), covering a wide range of values for the hyperparameter "alpha". This preliminary search helps to identify a suitable range for subsequent fine-tuning within the optimal parameter space. A 5-fold cross-validation strategy is used to optimize computational efficiency, which ensures robust model evaluation while reducing the risk of overfitting. The Root Mean Squared Error (RMSE) is used as a performance evaluation metric. 

In [13]:
def lasso_coarse_objective(trial):
    alpha = trial.suggest_float('alpha', 1e-9, 1e9, log=True)
    lasso = Lasso(alpha=alpha)
    kfolds = KFold(n_splits=5, shuffle=True, random_state=42)
    rmse_scores = cross_val_score(lasso, X_train, y_train, scoring='neg_root_mean_squared_error', cv=kfolds, n_jobs=-1)
    return -rmse_scores.mean()

In [14]:
lasso_coarse_study = optuna.create_study(direction='minimize')
lasso_coarse_study.optimize(lasso_coarse_objective, n_trials=20)
lasso_coarse_best_params = lasso_coarse_study.best_params
lasso_coarse_best_params

[I 2024-02-09 19:04:18,003] A new study created in memory with name: no-name-3a79e641-7114-40c5-878d-6b257df9fb34
[I 2024-02-09 19:04:28,295] Trial 0 finished with value: 0.7013207160161288 and parameters: {'alpha': 10243321.458660385}. Best is trial 0 with value: 0.7013207160161288.
[I 2024-02-09 19:08:33,305] Trial 1 finished with value: 0.16394805195471082 and parameters: {'alpha': 5.96865180528638e-07}. Best is trial 1 with value: 0.16394805195471082.
[I 2024-02-09 19:08:38,564] Trial 2 finished with value: 0.7013207160161288 and parameters: {'alpha': 401158.3420221274}. Best is trial 1 with value: 0.16394805195471082.
[I 2024-02-09 19:08:43,155] Trial 3 finished with value: 0.7013207160161288 and parameters: {'alpha': 836.8044150727266}. Best is trial 1 with value: 0.16394805195471082.
[I 2024-02-09 19:08:47,370] Trial 4 finished with value: 0.7013207160161288 and parameters: {'alpha': 9.170761659151827}. Best is trial 1 with value: 0.16394805195471082.
[I 2024-02-09 19:08:51,996]

{'alpha': 1.9516900194665794e-06}

The highest test scores were achieved within the alpha range of 10^-6, therefore, proceed with hyperparameter optimization, emphasizing this optimal parameter space.

In [15]:
def lasso_fine_objective(trial):
    alpha = trial.suggest_float('alpha', 1e-7, 1e-5, log=True)
    lasso = Lasso(alpha=alpha)
    kfolds = KFold(n_splits=5, shuffle=True, random_state=42)
    rmse_scores = cross_val_score(lasso, X_train, y_train, scoring='neg_root_mean_squared_error', cv=kfolds, n_jobs=-1)
    return -rmse_scores.mean()

In [16]:
lasso_fine_study = optuna.create_study(direction='minimize')
lasso_fine_study.optimize(lasso_fine_objective, n_trials=20)
lasso_fine_best_params = lasso_fine_study.best_params
lasso_fine_best_params

[I 2024-02-09 22:05:22,906] A new study created in memory with name: no-name-63e3e229-f876-40ed-8234-230ff6f3fbd4
[I 2024-02-09 22:09:08,328] Trial 0 finished with value: 0.16387247936991217 and parameters: {'alpha': 3.3649281346240794e-06}. Best is trial 0 with value: 0.16387247936991217.
[I 2024-02-09 22:12:05,381] Trial 1 finished with value: 0.1639890023075982 and parameters: {'alpha': 3.50098099799282e-07}. Best is trial 0 with value: 0.16387247936991217.
[I 2024-02-09 22:15:26,831] Trial 2 finished with value: 0.163960839806155 and parameters: {'alpha': 5.928976887895147e-06}. Best is trial 0 with value: 0.16387247936991217.
[I 2024-02-09 22:18:52,154] Trial 3 finished with value: 0.16397385111604876 and parameters: {'alpha': 6.14406310830036e-06}. Best is trial 0 with value: 0.16387247936991217.
[I 2024-02-09 22:22:16,639] Trial 4 finished with value: 0.1639914131708431 and parameters: {'alpha': 3.420306904942853e-07}. Best is trial 0 with value: 0.16387247936991217.
[I 2024-02-

{'alpha': 3.1599970196135527e-06}

Error is minimized at alpha = 3.1599970196135527e-06.

Train the model using the optimal hyperparameters, generate predictions for the test data using this trained model, evaluate the prediction performance, and display the results in a DataFrame. Additionally, save the fitted model as a pickle file, the model evaluation table, and the predictions as a CSV file.

In [17]:
lasso_model = Lasso(alpha=lasso_fine_best_params['alpha'])
lasso_start = datetime.now()
lasso_model.fit(X_train, y_train)
lasso_pred = lasso_model.predict(X_test)
lasso_stop = datetime.now()
lasso_delta = lasso_stop - lasso_start

c:\Sync\03_projects\data_science\projects\autoscout24-car-price-prediction\envs\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.949e+01, tolerance: 1.813e+00
  model = cd_fast.enet_coordinate_descent(


In [18]:
lasso_pred_inverse = np.expm1(lasso_pred)

lasso_r2 = r2_score(y_test, lasso_pred_inverse)
lasso_r2_adj = 1 - (1 - lasso_r2) * ((len(X_test) - 1) / (len(X_test) - len(X_test.columns) - 1))
lasso_rmse = np.sqrt(mean_squared_error(y_test, lasso_pred_inverse))
lasso_seconds = lasso_delta.seconds + lasso_delta.microseconds/1E6

lasso_evaluation = pd.DataFrame({
    'model': ['lasso'],
    'r2': [lasso_r2],
    'r2_adj': [lasso_r2_adj],
    'rmse': [lasso_rmse],
    'seconds': [lasso_seconds]
})

lasso_evaluation

,model,r2,r2_adj,rmse,seconds
0,lasso,0.949303,0.943652,4152.140941,53.182399


In [19]:
joblib.dump(lasso_model, '../models/models/lasso_model.pkl')
lasso_evaluation.to_csv(os.path.join("../models/evaluation/", "lasso_evaluation.csv"), index=False)
lasso_res = pd.DataFrame(lasso_pred)
lasso_res.index = X_test.index
lasso_res.columns = ["prediction"]
lasso_res.to_csv("../models/predictions/lasso_prediction.csv")

### 2.2.2. Ridge Regression

Start the ridge hyperparameter optimization with an initial rough exploration of the hyperparameter space using Optuna's TPESampler algorithm (Tree-Structured Parzen Estimator), covering a wide range of values for the hyperparameter "alpha". This preliminary search helps to identify a suitable range for subsequent fine-tuning within the optimal parameter space. A 5-fold cross-validation strategy is used to optimize computational efficiency, which ensures robust model evaluation while reducing the risk of overfitting. The Root Mean Squared Error (RMSE) is used as a performance evaluation metric. 

In [21]:
def ridge_coarse_objective(trial):
    alpha = trial.suggest_float('alpha', 1e-9, 1e9, log=True)
    ridge = Ridge(alpha=alpha)
    kfolds = KFold(n_splits=5, shuffle=True, random_state=42)
    rmse_scores = cross_val_score(ridge, X_train, y_train, scoring='neg_root_mean_squared_error', cv=kfolds, n_jobs=-1)
    return -rmse_scores.mean()

In [22]:
ridge_coarse_study = optuna.create_study(direction='minimize')
ridge_coarse_study.optimize(ridge_coarse_objective, n_trials=20)
ridge_coarse_best_params = ridge_coarse_study.best_params
ridge_coarse_best_params

[I 2024-02-09 23:36:40,634] A new study created in memory with name: no-name-980e92ef-e28b-4d0c-9bce-ce0fa5a6cbc8
[I 2024-02-09 23:36:50,021] Trial 0 finished with value: 0.16424857262649997 and parameters: {'alpha': 1.1743120410150176}. Best is trial 0 with value: 0.16424857262649997.
[I 2024-02-09 23:36:55,919] Trial 1 finished with value: 0.17098125868126882 and parameters: {'alpha': 10.535200575024227}. Best is trial 0 with value: 0.16424857262649997.
[I 2024-02-09 23:37:01,370] Trial 2 finished with value: 0.18931491451252808 and parameters: {'alpha': 49.63657417215814}. Best is trial 0 with value: 0.16424857262649997.
[I 2024-02-09 23:37:07,791] Trial 3 finished with value: 0.6998189954419571 and parameters: {'alpha': 3193783.064577547}. Best is trial 0 with value: 0.16424857262649997.
[I 2024-02-09 23:37:15,744] Trial 4 finished with value: 0.418584998793132 and parameters: {'alpha': 3237.982475531299}. Best is trial 0 with value: 0.16424857262649997.
[I 2024-02-09 23:37:21,331]

{'alpha': 0.350978657363417}

The highest test scores were achieved within the alpha range of 10^-1 to 10^0, therefore, proceed with hyperparameter optimization, emphasizing this optimal parameter space.

In [23]:
def ridge_fine_objective(trial):
    alpha = trial.suggest_float('alpha', 1e-1, 1e0, log=True)
    ridge = Ridge(alpha=alpha)
    kfolds = KFold(n_splits=5, shuffle=True, random_state=42)
    rmse_scores = cross_val_score(ridge, X_train, y_train, scoring='neg_root_mean_squared_error', cv=kfolds, n_jobs=-1)
    return -rmse_scores.mean()

In [24]:
ridge_fine_study = optuna.create_study(direction='minimize')
ridge_fine_study.optimize(ridge_fine_objective, n_trials=20)
ridge_fine_best_params = ridge_fine_study.best_params
ridge_fine_best_params

[I 2024-02-09 23:45:01,863] A new study created in memory with name: no-name-91b9b771-80b7-49b9-80a7-d1fbd13061b2
[I 2024-02-09 23:45:14,934] Trial 0 finished with value: 0.16400502975907394 and parameters: {'alpha': 0.7560482810597207}. Best is trial 0 with value: 0.16400502975907394.
[I 2024-02-09 23:45:22,776] Trial 1 finished with value: 0.16391336117257108 and parameters: {'alpha': 0.4982289691237173}. Best is trial 1 with value: 0.16391336117257108.
[I 2024-02-09 23:45:30,822] Trial 2 finished with value: 0.16389780865442807 and parameters: {'alpha': 0.38887340984488644}. Best is trial 2 with value: 0.16389780865442807.
[I 2024-02-09 23:45:36,991] Trial 3 finished with value: 0.16395869677401717 and parameters: {'alpha': 0.13233143481954387}. Best is trial 2 with value: 0.16389780865442807.
[I 2024-02-09 23:45:44,444] Trial 4 finished with value: 0.16405648400067147 and parameters: {'alpha': 0.8585941195554095}. Best is trial 2 with value: 0.16389780865442807.
[I 2024-02-09 23:45

{'alpha': 0.36415511361529757}

Error is minimized at alpha = 0.36415511361529757.

Train the ridge model using the optimal hyperparameters, generate predictions for the test data using this trained model, evaluate the prediction performance, and display the results in a DataFrame. Additionally, save the fitted model as a pickle file, the model evaluation table, and the predictions as a CSV file.

In [25]:
ridge_model = Ridge(alpha=ridge_fine_best_params['alpha'])
ridge_start = datetime.now()
ridge_model.fit(X_train, y_train)
ridge_pred = ridge_model.predict(X_test)
ridge_stop = datetime.now()
ridge_delta = ridge_stop - ridge_start

In [26]:
ridge_pred_inverse = np.expm1(ridge_pred)

ridge_r2 = r2_score(y_test, ridge_pred_inverse)
ridge_r2_adj = 1 - (1 - ridge_r2) * ((len(X_test) - 1) / (len(X_test) - len(X_test.columns) - 1))
ridge_rmse = np.sqrt(mean_squared_error(y_test, ridge_pred_inverse))
ridge_seconds = ridge_delta.seconds + ridge_delta.microseconds/1E6

ridge_evaluation = pd.DataFrame({
    'model': ['ridge'],
    'r2': [ridge_r2],
    'r2_adj': [ridge_r2_adj],
    'rmse': [ridge_rmse],
    'seconds': [ridge_seconds]
})

ridge_evaluation

,model,r2,r2_adj,rmse,seconds
0,ridge,0.940112,0.933437,4512.857727,1.733712


In [27]:
joblib.dump(ridge_model, '../models/models/ridge_model.pkl')
ridge_evaluation.to_csv(os.path.join("../models/evaluation/", "ridge_evaluation.csv"), index=False)
ridge_res = pd.DataFrame(ridge_pred)
ridge_res.index = X_test.index
ridge_res.columns = ["prediction"]
ridge_res.to_csv("../models/predictions/ridge_prediction.csv")

### 2.2.3. Elastic Net Regression

Start the elastic net hyperparameter optimization with an initial rough exploration of the hyperparameter space using Optuna's TPESampler algorithm (Tree-Structured Parzen Estimator), covering a wide range of values for the hyperparameters "alpha" and the "l1 ratio". This preliminary search helps to identify a suitable range for subsequent fine-tuning within the optimal parameter space. A 5-fold cross-validation strategy is used to optimize computational efficiency, which ensures robust model evaluation while reducing the risk of overfitting. The Root Mean Squared Error (RMSE) is used as a performance evaluation metric. 

In [32]:
def elasticnet_coarse_objective(trial):
    alpha = trial.suggest_float('alpha', 1e-9, 1e9, log=True)
    l1_ratio = trial.suggest_float('l1_ratio', 0, 1)
    elasticnet = ElasticNet(alpha=alpha, l1_ratio=l1_ratio)
    kfolds = KFold(n_splits=5, shuffle=True, random_state=42)
    rmse_scores = cross_val_score(elasticnet, X_train, y_train, scoring='neg_root_mean_squared_error', cv=kfolds, n_jobs=-1)
    return -rmse_scores.mean()

In [33]:
elasticnet_coarse_study = optuna.create_study(direction='minimize')
elasticnet_coarse_study.optimize(elasticnet_coarse_objective, n_trials=20)
elasticnet_coarse_best_params = elasticnet_coarse_study.best_params
elasticnet_coarse_best_params

[I 2024-02-09 23:56:44,342] A new study created in memory with name: no-name-6bcc7919-3a04-4ee0-9f52-ca1229989486
[I 2024-02-09 23:56:51,473] Trial 0 finished with value: 0.7013207160161288 and parameters: {'alpha': 73.05249792172569, 'l1_ratio': 0.48929492242600703}. Best is trial 0 with value: 0.7013207160161288.
[I 2024-02-10 00:00:09,481] Trial 1 finished with value: 0.16428393249434645 and parameters: {'alpha': 3.698522250451831e-09, 'l1_ratio': 0.8038315597375395}. Best is trial 1 with value: 0.16428393249434645.
[I 2024-02-10 00:00:13,032] Trial 2 finished with value: 0.7013207160161288 and parameters: {'alpha': 1.5506251802243947, 'l1_ratio': 0.7408186330293501}. Best is trial 1 with value: 0.16428393249434645.
[I 2024-02-10 00:00:20,307] Trial 3 finished with value: 0.19518038543879007 and parameters: {'alpha': 0.0009063432259673031, 'l1_ratio': 0.3853263311447038}. Best is trial 1 with value: 0.16428393249434645.
[I 2024-02-10 00:03:35,356] Trial 4 finished with value: 0.1641

{'alpha': 2.430224145742283e-06, 'l1_ratio': 0.26693419316393463}

The highest test scores were achieved within the alpha range of 10^-6 and l1 ratio 10^-1 to 10^0, therefore, proceed with hyperparameter optimization, emphasizing this optimal parameter space.

In [36]:
def elasticnet_fine_objective(trial):
    alpha = trial.suggest_float('alpha', 1e-7, 1e-5, log=True)
    l1_ratio = trial.suggest_float('l1_ratio', 0.1, 1)
    elasticnet = ElasticNet(alpha=alpha, l1_ratio=l1_ratio)
    kfolds = KFold(n_splits=5, shuffle=True, random_state=42)
    rmse_scores = cross_val_score(elasticnet, X_train, y_train, scoring='neg_root_mean_squared_error', cv=kfolds, n_jobs=-1)
    return -rmse_scores.mean()

In [37]:
elasticnet_fine_study = optuna.create_study(direction='minimize')
elasticnet_fine_study.optimize(elasticnet_fine_objective, n_trials=20)
elasticnet_fine_best_params = elasticnet_fine_study.best_params
elasticnet_fine_best_params

[I 2024-02-10 09:23:34,290] A new study created in memory with name: no-name-4cba7b1c-2cca-4da3-af2e-fa5f5c067ab6
[I 2024-02-10 09:26:20,845] Trial 0 finished with value: 0.16418770947259687 and parameters: {'alpha': 1.0611453153664617e-07, 'l1_ratio': 0.45916353635923446}. Best is trial 0 with value: 0.16418770947259687.
[I 2024-02-10 09:28:53,523] Trial 1 finished with value: 0.16424038449077666 and parameters: {'alpha': 1.1420750225616437e-07, 'l1_ratio': 0.20522381933850875}. Best is trial 0 with value: 0.16418770947259687.
[I 2024-02-10 09:31:25,821] Trial 2 finished with value: 0.164071997567935 and parameters: {'alpha': 2.2423122236687345e-07, 'l1_ratio': 0.7992642642371942}. Best is trial 2 with value: 0.164071997567935.
[I 2024-02-10 09:34:30,205] Trial 3 finished with value: 0.163878654344104 and parameters: {'alpha': 2.7176833107958028e-06, 'l1_ratio': 0.8533991235497324}. Best is trial 3 with value: 0.163878654344104.
[I 2024-02-10 09:37:28,901] Trial 4 finished with value:

{'alpha': 3.943231695230683e-06, 'l1_ratio': 0.6838802956392738}

Error is minimized at alpha = 3.943231695230683e-06 and l1 ratio = 0.6838802956392738.

Train the elastic net model using the optimal hyperparameters, generate predictions for the test data using this trained model, evaluate the prediction performance, and display the results in a DataFrame. Additionally, save the fitted model as a pickle file, the model evaluation table, and the predictions as a CSV file.

In [38]:
elasticnet_model = ElasticNet(alpha=elasticnet_fine_best_params['alpha'], l1_ratio=elasticnet_fine_best_params['l1_ratio'])
elasticnet_start = datetime.now()
elasticnet_model.fit(X_train, y_train)
elasticnet_pred = elasticnet_model.predict(X_test)
elasticnet_stop = datetime.now()
elasticnet_delta = elasticnet_stop - elasticnet_start

c:\Sync\03_projects\data_science\projects\autoscout24-car-price-prediction\envs\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.988e+01, tolerance: 1.813e+00
  model = cd_fast.enet_coordinate_descent(


In [39]:
elasticnet_pred_inverse = np.expm1(elasticnet_pred)

elasticnet_r2 = r2_score(y_test, elasticnet_pred_inverse)
elasticnet_r2_adj = 1 - (1 - elasticnet_r2) * ((len(X_test) - 1) / (len(X_test) - len(X_test.columns) - 1))
elasticnet_rmse = np.sqrt(mean_squared_error(y_test, elasticnet_pred_inverse))
elasticnet_seconds = elasticnet_delta.seconds + elasticnet_delta.microseconds/1E6

elasticnet_evaluation = pd.DataFrame({
    'model': ['elasticnet'],
    'r2': [elasticnet_r2],
    'r2_adj': [elasticnet_r2_adj],
    'rmse': [elasticnet_rmse],
    'seconds': [elasticnet_seconds]
})

elasticnet_evaluation

,model,r2,r2_adj,rmse,seconds
0,elasticnet,0.948092,0.942306,4201.461942,77.137834


In [40]:
joblib.dump(elasticnet_model, '../models/models/elasticnet_model.pkl')
elasticnet_evaluation.to_csv(os.path.join("../models/evaluation/", "elasticnet_evaluation.csv"), index=False)
elasticnet_res = pd.DataFrame(elasticnet_pred)
elasticnet_res.index = X_test.index
elasticnet_res.columns = ["prediction"]
elasticnet_res.to_csv("../models/predictions/elasticnet_prediction.csv")

## 2.3. Gaussian Process Regression

## 2.4. Bayesian Linear Regression

## 2.5. Robust Regression

### 2.5.1. Huber Regression

Start the huber regression hyperparameter optimization with an initial rough exploration of the hyperparameter space using Optuna's TPESampler algorithm (Tree-Structured Parzen Estimator), covering a wide range of values for the hyperparameters "alpha" and "epsilon". This preliminary search helps to identify a suitable range for subsequent fine-tuning within the optimal parameter space. A 5-fold cross-validation strategy is used to optimize computational efficiency, which ensures robust model evaluation while reducing the risk of overfitting. The Root Mean Squared Error (RMSE) is used as a performance evaluation metric. 

In [54]:
def huber_coarse_objective(trial):
    alpha = trial.suggest_float('alpha', 1e-9, 1e9, log=True)
    epsilon = trial.suggest_float('epsilon', 1.0, 2.0)
    huber = HuberRegressor(alpha=alpha, epsilon=epsilon)
    kfolds = KFold(n_splits=5, shuffle=True, random_state=42)
    rmse_scores = cross_val_score(huber, X_train, y_train, scoring='neg_root_mean_squared_error', cv=kfolds, n_jobs=-1)
    return -rmse_scores.mean()

In [55]:
huber_coarse_study = optuna.create_study(direction='minimize')
huber_coarse_study.optimize(huber_coarse_objective, n_trials=20)
huber_coarse_best_params = huber_coarse_study.best_params
huber_coarse_best_params

[I 2024-02-10 15:47:23,505] A new study created in memory with name: no-name-47385cf4-8fe6-4b75-81f1-82d2eb27299f
[I 2024-02-10 15:49:30,618] Trial 0 finished with value: 0.17015942234510364 and parameters: {'alpha': 0.2774699984085081, 'epsilon': 1.519249436181226}. Best is trial 0 with value: 0.17015942234510364.
[I 2024-02-10 15:51:30,413] Trial 1 finished with value: 0.1714768544964337 and parameters: {'alpha': 9.775799499879854e-08, 'epsilon': 1.2290931912935026}. Best is trial 0 with value: 0.17015942234510364.
[I 2024-02-10 15:53:41,531] Trial 2 finished with value: 0.16982439214150563 and parameters: {'alpha': 0.22676287042545554, 'epsilon': 1.8267688394701462}. Best is trial 2 with value: 0.16982439214150563.
[I 2024-02-10 15:55:54,629] Trial 3 finished with value: 0.17027320860399597 and parameters: {'alpha': 3.1321014496836833e-07, 'epsilon': 1.7555549734712543}. Best is trial 2 with value: 0.16982439214150563.
[I 2024-02-10 15:58:04,948] Trial 4 finished with value: 0.17030

{'alpha': 7.37950301489792e-05, 'epsilon': 1.9886807708950724}

The highest test scores were achieved within the alpha range of 10^-6 and epsilon near 2, therefore, proceed with hyperparameter optimization, emphasizing this optimal parameter space.

In [45]:
def huber_fine_objective(trial):
    alpha = trial.suggest_float('alpha', 1e-7, 1e-5, log=True)
    epsilon = trial.suggest_float('epsilon', 1.9, 2.0)
    huber = HuberRegressor(alpha=alpha, epsilon=epsilon)
    kfolds = KFold(n_splits=5, shuffle=True, random_state=42)
    rmse_scores = cross_val_score(huber, X_train, y_train, scoring='neg_root_mean_squared_error', cv=kfolds, n_jobs=-1)
    return -rmse_scores.mean()

In [46]:
huber_fine_study = optuna.create_study(direction='minimize')
huber_fine_study.optimize(huber_fine_objective, n_trials=20)
huber_fine_best_params = huber_fine_study.best_params
huber_fine_best_params

[I 2024-02-10 14:25:45,437] A new study created in memory with name: no-name-277db13d-15d2-452b-91f4-05b591138b2a
[I 2024-02-10 14:28:23,261] Trial 0 finished with value: 0.16946093642638044 and parameters: {'alpha': 2.102586098773178e-07, 'epsilon': 1.9317520120367464}. Best is trial 0 with value: 0.16946093642638044.
[I 2024-02-10 14:30:28,051] Trial 1 finished with value: 0.16974317582031384 and parameters: {'alpha': 1.9436391916273176e-07, 'epsilon': 1.9107556093185707}. Best is trial 0 with value: 0.16946093642638044.
[I 2024-02-10 14:32:34,315] Trial 2 finished with value: 0.16953010917304015 and parameters: {'alpha': 2.301815405272179e-06, 'epsilon': 1.905033163742058}. Best is trial 0 with value: 0.16946093642638044.
[I 2024-02-10 14:34:33,496] Trial 3 finished with value: 0.16966976655452734 and parameters: {'alpha': 3.356769436674778e-06, 'epsilon': 1.9473879754958292}. Best is trial 0 with value: 0.16946093642638044.
[I 2024-02-10 14:36:35,010] Trial 4 finished with value: 0

{'alpha': 8.627070834562276e-06, 'epsilon': 1.995917458517765}

In [49]:
huber_fine_best_params

{'alpha': 8.627070834562276e-06, 'epsilon': 1.995917458517765}

Error is minimized at alpha = 8.627070834562276e-06 and epsilon = 1.995917458517765.

Train the Huber regression model using the optimal hyperparameters, generate predictions for the test data using this trained model, evaluate the prediction performance, and display the results in a DataFrame. Additionally, save the fit-ted model as a pickle file, the model evaluation table, and the predictions as a CSV file.

In [68]:
huber_model = HuberRegressor(alpha=huber_fine_best_params['alpha'], epsilon=huber_fine_best_params['epsilon'], max_iter=10000)
huber_start = datetime.now()
huber_model.fit(X_train, y_train.values.ravel())
huber_pred = huber_model.predict(X_test)
huber_stop = datetime.now()
huber_delta = huber_stop - huber_start

In [69]:
huber_pred_inverse = np.expm1(huber_pred)

huber_r2 = r2_score(y_test, huber_pred_inverse)
huber_r2_adj = 1 - (1 - huber_r2) * ((len(X_test) - 1) / (len(X_test) - len(X_test.columns) - 1))
huber_rmse = np.sqrt(mean_squared_error(y_test, huber_pred_inverse))
huber_seconds = huber_delta.seconds + huber_delta.microseconds/1E6

huber_evaluation = pd.DataFrame({
    'model': ['huber'],
    'r2': [huber_r2],
    'r2_adj': [huber_r2_adj],
    'rmse': [huber_rmse],
    'seconds': [huber_seconds]
})

huber_evaluation

,model,r2,r2_adj,rmse,seconds
0,huber,0.950388,0.944858,4107.468443,1056.937912


In [70]:
joblib.dump(huber_model, '../models/models/huber_model.pkl')
huber_evaluation.to_csv(os.path.join("../models/evaluation/", "huber_evaluation.csv"), index=False)
huber_res = pd.DataFrame(huber_pred)
huber_res.index = X_test.index
huber_res.columns = ["prediction"]
huber_res.to_csv("../models/predictions/huber_prediction.csv")

### 2.5.2. Quantile Regression

Start the Quantile regression hyperparameter optimization with an initial rough exploration of the hyperparameter space using Optuna's TPESampler algorithm (Tree-Structured Parzen Estimator), covering a wide range of values for the hyperparameters "alpha" and "epsilon". This preliminary search helps to identify a suitable range for subsequent fine-tuning within the optimal parameter space. A 5-fold cross-validation strategy is used to optimize computational efficiency, which ensures robust model evaluation while reducing the risk of overfitting. The Root Mean Squared Error (RMSE) is used as a performance evaluation metric. 

In [72]:
def quantile_coarse_objective(trial):
    alpha = trial.suggest_float('alpha', 1e-9, 1e9, log=True)
    quantile = trial.suggest_float('quantile', 0, 1.0)
    quantile_regressor = QuantileRegressor(alpha=alpha, quantile=quantile)
    kfolds = KFold(n_splits=5, shuffle=True, random_state=42)
    rmse_scores = cross_val_score(quantile_regressor, X_train, y_train, scoring='neg_root_mean_squared_error', cv=kfolds, n_jobs=-1)
    return -rmse_scores.mean()

In [74]:
quantile_coarse_study = optuna.create_study(direction='minimize')
quantile_coarse_study.optimize(quantile_coarse_objective, n_trials=20)
quantile_coarse_best_params = quantile_coarse_study.best_params
quantile_coarse_best_params

[I 2024-02-10 19:48:21,322] A new study created in memory with name: no-name-44941ee8-d115-4e8a-a87a-071283512174
[I 2024-02-10 19:52:14,654] Trial 0 finished with value: 0.1972568877155163 and parameters: {'alpha': 1.484268718308485e-05, 'quantile': 0.22936699901226265}. Best is trial 0 with value: 0.1972568877155163.
[I 2024-02-10 19:53:41,543] Trial 1 finished with value: 0.8930991022683878 and parameters: {'alpha': 13497871.075306997, 'quantile': 0.7816180825835011}. Best is trial 0 with value: 0.1972568877155163.
[I 2024-02-10 19:57:09,497] Trial 2 finished with value: 0.16749112269001215 and parameters: {'alpha': 4.302528127718235e-06, 'quantile': 0.5536598113738292}. Best is trial 2 with value: 0.16749112269001215.
[I 2024-02-10 19:58:14,089] Trial 3 finished with value: 1.1664174837436694 and parameters: {'alpha': 0.17266249627736224, 'quantile': 0.058166990855073486}. Best is trial 2 with value: 0.16749112269001215.
[I 2024-02-10 19:59:15,775] Trial 4 finished with value: 1.43

{'alpha': 2.5148222550759484e-06, 'quantile': 0.472165721106035}

The highest test scores were achieved within the alpha range of 10^-6 and quantile near 50%, therefore, proceed with hyperparameter optimization, emphasizing this optimal parameter space.

In [75]:
def quantile_fine_objective(trial):
    alpha = trial.suggest_float('alpha', 1e-7, 1e-5, log=True)
    quantile = trial.suggest_float('quantile', 0.4, 0.6)
    quantile_regressor = QuantileRegressor(alpha=alpha, quantile=quantile)
    kfolds = KFold(n_splits=5, shuffle=True, random_state=42)
    rmse_scores = cross_val_score(quantile_regressor, X_train, y_train, scoring='neg_root_mean_squared_error', cv=kfolds, n_jobs=-1)
    return -rmse_scores.mean()

In [76]:
quantile_fine_study = optuna.create_study(direction='minimize')
quantile_fine_study.optimize(quantile_fine_objective, n_trials=20)
quantile_fine_best_params = quantile_fine_study.best_params
quantile_fine_best_params

[I 2024-02-10 21:18:57,987] A new study created in memory with name: no-name-44f01fe2-420c-4b75-acc5-7a64e4feb4ef
[I 2024-02-10 21:22:39,238] Trial 0 finished with value: 0.1689371778717976 and parameters: {'alpha': 7.113259094226004e-06, 'quantile': 0.5773948560320911}. Best is trial 0 with value: 0.1689371778717976.
[I 2024-02-10 21:26:09,335] Trial 1 finished with value: 0.16542629369480316 and parameters: {'alpha': 1.1264750222011006e-06, 'quantile': 0.5030638204633192}. Best is trial 1 with value: 0.16542629369480316.
[I 2024-02-10 21:29:45,033] Trial 2 finished with value: 0.16593816380258478 and parameters: {'alpha': 8.917089108907226e-07, 'quantile': 0.4563644388645848}. Best is trial 1 with value: 0.16542629369480316.
[I 2024-02-10 21:33:18,550] Trial 3 finished with value: 0.16567255173782647 and parameters: {'alpha': 3.5417794742522604e-06, 'quantile': 0.46351084921808444}. Best is trial 1 with value: 0.16542629369480316.
[I 2024-02-10 21:36:48,664] Trial 4 finished with val

{'alpha': 1.1264750222011006e-06, 'quantile': 0.5030638204633192}

Error is minimized at alpha = 1.1264750222011006e-06 and quantile = 0.5030638204633192.

Train the quantile regression model using the optimal hyperparameters, generate predictions for the test data using this trained model, evaluate the prediction performance, and display the results in a DataFrame. Additionally, save the fit-ted model as a pickle file, the model evaluation table, and the predictions as a CSV file.

In [78]:
quantile_model = QuantileRegressor(alpha=quantile_fine_best_params['alpha'], quantile=quantile_fine_best_params['quantile'])
quantile_start = datetime.now()
quantile_model.fit(X_train, y_train)
quantile_pred = quantile_model.predict(X_test)
quantile_stop = datetime.now()
quantile_delta = quantile_stop - quantile_start

c:\Sync\03_projects\data_science\projects\autoscout24-car-price-prediction\envs\Lib\site-packages\sklearn\utils\validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [79]:
quantile_pred_inverse = np.expm1(quantile_pred)

quantile_r2 = r2_score(y_test, quantile_pred_inverse)
quantile_r2_adj = 1 - (1 - quantile_r2) * ((len(X_test) - 1) / (len(X_test) - len(X_test.columns) - 1))
quantile_rmse = np.sqrt(mean_squared_error(y_test, quantile_pred_inverse))
quantile_seconds = quantile_delta.seconds + quantile_delta.microseconds/1E6

quantile_evaluation = pd.DataFrame({
    'model': ['quantile'],
    'r2': [quantile_r2],
    'r2_adj': [quantile_r2_adj],
    'rmse': [quantile_rmse],
    'seconds': [quantile_seconds]
})

quantile_evaluation

,model,r2,r2_adj,rmse,seconds
0,quantile,0.952018,0.94667,4039.448262,110.754906


In [80]:
joblib.dump(quantile_model, '../models/models/quantile_model.pkl')
quantile_evaluation.to_csv(os.path.join("../models/evaluation/", "quantile_evaluation.csv"), index=False)
quantile_res = pd.DataFrame(quantile_pred)
quantile_res.index = X_test.index
quantile_res.columns = ["prediction"]
quantile_res.to_csv("../models/predictions/quantile_prediction.csv")

### 2.5.3. RANSAC Regression

### 2.5.4. Theil Sen Regression

## 2.6. K-Nearest Neighbors Regression

Start the k-Nearest Neighbors Regression (KNN) (hyper)parameter optimization with an initial rough exploration of the hyperparameter space using Optuna's TPESampler algorithm (Tree-Structured Parzen Estimator), covering a wide range of values for the hyperparameter "n_neighbors", "metric", "weights". This preliminary search helps to identify a suitable range for subsequent fine-tuning within the optimal parameter space. A 5-fold cross-validation strategy is used to optimize computational efficiency, which ensures robust model evaluation while reducing the risk of overfitting. The Root Mean Squared Error (RMSE) is used as a performance evaluation metric. 

In [15]:
def knn_coarse_objective(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 1, 100)
    metric = trial.suggest_categorical('metric', ['euclidean', 'manhattan', 'cosine'])
    weights = trial.suggest_categorical('weights', ['uniform', 'distance'])
    knn = KNeighborsRegressor(n_neighbors=n_neighbors, metric=metric, weights=weights)
    kfolds = KFold(n_splits=5, shuffle=True, random_state=42)
    rmse_scores = cross_val_score(knn, X_train, y_train, scoring='neg_root_mean_squared_error', cv=kfolds, n_jobs=1)
    return -rmse_scores.mean()

In [16]:
knn_coarse_study = optuna.create_study(direction='minimize')
knn_coarse_study.optimize(knn_coarse_objective, n_trials=20)
knn_coarse_best_params = knn_coarse_study.best_params
knn_coarse_best_params

[I 2024-02-12 12:51:40,678] A new study created in memory with name: no-name-c44fdcdb-d356-4703-96a7-bfb74ed1bfc2
[I 2024-02-12 12:53:49,544] Trial 0 finished with value: 0.2261274055871673 and parameters: {'n_neighbors': 95, 'metric': 'euclidean', 'weights': 'distance'}. Best is trial 0 with value: 0.2261274055871673.
[I 2024-02-12 12:55:04,637] Trial 1 finished with value: 0.2091143201231799 and parameters: {'n_neighbors': 51, 'metric': 'euclidean', 'weights': 'distance'}. Best is trial 1 with value: 0.2091143201231799.
[I 2024-02-12 12:55:53,572] Trial 2 finished with value: 0.19542551131956953 and parameters: {'n_neighbors': 6, 'metric': 'euclidean', 'weights': 'uniform'}. Best is trial 2 with value: 0.19542551131956953.
[I 2024-02-12 12:56:57,028] Trial 3 finished with value: 0.18298070254262128 and parameters: {'n_neighbors': 8, 'metric': 'cosine', 'weights': 'distance'}. Best is trial 3 with value: 0.18298070254262128.
[I 2024-02-12 13:11:25,748] Trial 4 finished with value: 0.2

{'n_neighbors': 8, 'metric': 'cosine', 'weights': 'distance'}

The highest test scores were achieved at  n_neighbors < 25, a metric of either cosine or manhattan, and weights based on distance. Therefore, proceed with hyperparameter optimization, emphasizing this optimal parameter space.

In [17]:
def knn_fine_objective(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 1, 25)
    metric = trial.suggest_categorical('metric', ['manhattan', 'cosine'])
    weights = trial.suggest_categorical('weights', ['distance'])
    knn = KNeighborsRegressor(n_neighbors=n_neighbors, metric=metric, weights=weights)
    kfolds = KFold(n_splits=5, shuffle=True, random_state=42)
    rmse_scores = cross_val_score(knn, X_train, y_train, scoring='neg_root_mean_squared_error', cv=kfolds, n_jobs=1)
    return -rmse_scores.mean()

In [18]:
knn_fine_study = optuna.create_study(direction='minimize')
knn_fine_study.optimize(knn_fine_objective, n_trials=20)
knn_fine_best_params = knn_fine_study.best_params
knn_fine_best_params

[I 2024-02-12 14:20:42,627] A new study created in memory with name: no-name-690b2630-1c27-44bb-b8e4-7aec5666ce27
[I 2024-02-12 14:21:50,264] Trial 0 finished with value: 0.18301881387123556 and parameters: {'n_neighbors': 9, 'metric': 'cosine', 'weights': 'distance'}. Best is trial 0 with value: 0.18301881387123556.
[I 2024-02-12 14:38:46,774] Trial 1 finished with value: 0.18976022309971669 and parameters: {'n_neighbors': 23, 'metric': 'manhattan', 'weights': 'distance'}. Best is trial 0 with value: 0.18301881387123556.
[I 2024-02-12 14:39:42,554] Trial 2 finished with value: 0.19082744487686298 and parameters: {'n_neighbors': 2, 'metric': 'cosine', 'weights': 'distance'}. Best is trial 0 with value: 0.18301881387123556.
[I 2024-02-12 14:54:17,713] Trial 3 finished with value: 0.190884885222314 and parameters: {'n_neighbors': 25, 'metric': 'manhattan', 'weights': 'distance'}. Best is trial 0 with value: 0.18301881387123556.
[I 2024-02-12 15:08:49,199] Trial 4 finished with value: 0.1

{'n_neighbors': 5, 'metric': 'manhattan', 'weights': 'distance'}

Error is minimized at 'n_neighbors' = 5, 'metric' = 'manhattan' and 'weights' = 'distance'

Train the knn regression model using the optimal hyperparameters, generate predictions for the test data using this trained model, evaluate the prediction performance, and display the results in a DataFrame. Additionally, save the fitted model as a pickle file, the model evaluation table, and the predictions as a CSV file.

In [21]:
knn_model = KNeighborsRegressor(n_neighbors=knn_fine_best_params['n_neighbors'], 
                                metric=knn_fine_best_params['metric'],
                                weights=knn_fine_best_params['weights'])
knn_start = datetime.now()
knn_model.fit(X_train, y_train)
knn_pred = knn_model.predict(X_test)
knn_stop = datetime.now()
knn_delta = knn_stop - knn_start

In [22]:
knn_pred_inverse = np.expm1(knn_pred)

knn_r2 = r2_score(y_test, knn_pred_inverse)
knn_r2_adj = 1 - (1 - knn_r2) * ((len(X_test) - 1) / (len(X_test) - len(X_test.columns) - 1))
knn_rmse = np.sqrt(mean_squared_error(y_test, knn_pred_inverse))
knn_seconds = knn_delta.seconds + knn_delta.microseconds/1E6

knn_evaluation = pd.DataFrame({
    'model': ['knn'],
    'r2': [knn_r2],
    'r2_adj': [knn_r2_adj],
    'rmse': [knn_rmse],
    'seconds': [knn_seconds]
})
knn_evaluation

,model,r2,r2_adj,rmse,seconds
0,knn,0.904055,0.893361,5712.046169,257.094118


In [23]:
joblib.dump(knn_model, '../models/models/knn_model.pkl')
knn_evaluation.to_csv(os.path.join("../models/evaluation/", "knn_evaluation.csv"), index=False)
knn_res = pd.DataFrame(knn_pred)
knn_res.index = X_test.index
knn_res.columns = ["prediction"]
knn_res.to_csv("../models/predictions/knn_prediction.csv")

## 2.7. Artificial Neural Networks

### 2.7.1. Multi-Layer Perceptron Regressor

## 2.8. Support Vector Regression

Start the Support Vector Regression (SVR) hyperparameter optimization with an initial rough exploration of the hyperparameter space using Optuna's TPESampler algorithm (Tree-Structured Parzen Estimator), covering a wide range of values for the hyperparameter "C", "gamma", "epsilon" and "kernel". This preliminary search helps to identify a suitable range for subsequent fine-tuning within the optimal parameter space. A 5-fold cross-validation strategy is used to optimize computational efficiency, which ensures robust model evaluation while reducing the risk of overfitting. The Root Mean Squared Error (RMSE) is used as a performance evaluation metric. 

In [64]:
def svr_coarse_objective(trial):
    C = trial.suggest_float('C', 1e-6, 1e3, log=True)
    tol = trial.suggest_float("tol", 1e-9, 1e-1, log=True)
    epsilon = trial.suggest_float("epsilon", 1e-3, 1e1, log=True)
    loss = trial.suggest_categorical("loss", ['epsilon_insensitive', 'squared_epsilon_insensitive'])
    svr = LinearSVR(C=C, loss=loss, tol=tol, epsilon=epsilon)
    kfolds = KFold(n_splits=5, shuffle=True, random_state=42)
    rmse_scores = cross_val_score(svr, X_train, y_train, scoring='neg_root_mean_squared_error', cv=kfolds, n_jobs=-1)
    return -rmse_scores.mean()

In [66]:
svr_coarse_study = optuna.create_study(direction='minimize')
svr_coarse_study.optimize(svr_coarse_objective, n_trials=20)
svr_coarse_best_params = svr_coarse_study.best_params
svr_coarse_best_params

[I 2024-02-14 20:53:12,579] A new study created in memory with name: no-name-68a1573b-1c66-4b67-9685-c7c6b8b3491e
[I 2024-02-14 20:53:35,772] Trial 0 finished with value: 3.1052196142578756 and parameters: {'C': 1.3683239321714658e-05, 'tol': 2.0625339754568704e-09, 'epsilon': 0.010674442309464882, 'loss': 'squared_epsilon_insensitive'}. Best is trial 0 with value: 3.1052196142578756.
[I 2024-02-14 20:53:44,409] Trial 1 finished with value: 0.16474006267580193 and parameters: {'C': 6.291914240097194, 'tol': 0.064427774518623, 'epsilon': 0.05692119564645714, 'loss': 'squared_epsilon_insensitive'}. Best is trial 1 with value: 0.16474006267580193.
[I 2024-02-14 20:53:48,779] Trial 2 finished with value: 0.19697368771351925 and parameters: {'C': 0.016482491312162464, 'tol': 5.687953353024018e-05, 'epsilon': 0.07245354574398293, 'loss': 'squared_epsilon_insensitive'}. Best is trial 1 with value: 0.16474006267580193.
[I 2024-02-14 20:53:53,594] Trial 3 finished with value: 0.1700282861285248

{'C': 1.2972909167443012,
 'tol': 7.231119513167499e-07,
 'epsilon': 0.021282355887059613,
 'loss': 'squared_epsilon_insensitive'}

The highest test scores were achieved within range of 
10^-1 and 10^2 for "C", 
10^-7 and 10^-2 for "tol", 
10^-3 and 10^-1 for "epsilon" and
a "squared_epsilon_insensitive" type "loss, 
therefore, proceed with hyperparameter optimization, emphasizing this optimal parameter space.

## 2.9. Decision Trees Regression

## 2.10. Ensemble

### 2.10.1. Ada Boost Regressor

### 2.10.2. Bagging Regressor

### 2.10.3. Extra Tree Regressor

### 2.10.4. Gradient Boosting Regressor

### 2.10.5. XGBoost Regressor

### 2.10.6. LightGBM Regressor

### 2.10.7. Random Forest Regressor

### 2.10.8. Extra Trees Regressor

### 2.10.9. Stacking Regressor

### 2.10.10. Voting Regressor

### 2.10.11. Histogram-based Gradient Boosting Regressor

## 2.10. Dimensionality-Reduced Regression